# **Quantum Finance: Portfolio Optimization with QAOA**

This notebook demonstrates how to formulate and solve a portfolio optimization problem using the Quantum Approximate Optimization Algorithm (QAOA) on Qiskit.

## **1. Data Acquisition and Preprocessing**

We start by fetching historical stock data, calculating daily returns, mean returns, and the covariance matrix. These will be essential for constructing our optimization problem.

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import yfinance as yf

In [2]:
# List of 10 real stocks (modify as you wish)
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA',
           'NVDA', 'META', 'JPM', 'V', 'NFLX']

print("Downloading historical data...")
data = yf.download(tickers, period='1y')

# 2. Extract close prices (multi-index safe)
if isinstance(data.columns, pd.MultiIndex):
    close_prices = data['Close']
else:
    close_prices = data

# Drop columns with too many missing values (e.g., >10%)
close_prices = close_prices.dropna(axis=1, thresh=len(close_prices) * 0.9)
selected_tickers = close_prices.columns.tolist()
print(f"✅ Final selected assets ({len(selected_tickers)}): {selected_tickers}")

# 3. Daily returns
returns = close_prices.pct_change().dropna()
mean_returns = returns.mean().values
cov_matrix = returns.cov().values

n = len(selected_tickers)
print(f"\nNumber of assets: {n}")

[*********************100%***********************]  10 of 10 completed

✅ Final selected assets (10): ['AAPL', 'AMZN', 'GOOGL', 'JPM', 'META', 'MSFT', 'NFLX', 'NVDA', 'TSLA', 'V']

Number of assets: 10


## **2. Problem Formulation: Quadratic Unconstrained Binary Optimization (QUBO)**

Portfolio optimization can be formulated as a QUBO problem. We aim to minimize a function that balances risk and return. The decision variables $x_i$ are binary, where $x_i=1$ if asset $i$ is selected and $x_i=0$ otherwise.

The general form of the QUBO objective function is:

$$ \text{Minimize: } H(x) = \sum_{i} Q_{ii} x_i + \sum_{i<j} Q_{ij} x_i x_j $$

For portfolio optimization, we typically use a mean-variance objective, which seeks to minimize the portfolio variance while maximizing its expected return. This can be expressed as:

$$ \text{Minimize: } \quad \lambda \sum_{i,j} \Sigma_{ij} x_i x_j - \sum_i \mu_i x_i $$

Where:
* $x_i \in \{0, 1\}$ are the binary decision variables.
* $\Sigma_{ij}$ are the elements of the covariance matrix, representing risk.
* $\mu_i$ are the expected returns for each asset.
* $\lambda$ is a positive risk-aversion parameter, controlling the trade-off between risk and return.

### **2.1 Encoding Risk Term (Variance)**

The risk term corresponds to the portfolio variance:

$$ \text{Risk Term} = \lambda \sum_{i,j} \Sigma_{ij} x_i x_j $$

This term contributes to the quadratic coefficients ($Q_{ij}$) in our QUBO, encouraging diversification and penalizing high variance.

In [3]:
# Build Risk QUBO (λ Σ)
lambda_risk = 10.0  # Penalty weight (tune this) - Higher lambda means more risk aversion
Q_risk = {}
for i in range(n):
    for j in range(n):
        Q_risk[(i, j)] = lambda_risk * cov_matrix[i][j]

In [4]:
# Build Return QUBO (-μ)
Q_return = {}
for i in range(n):
    Q_return[(i, i)] = -mean_returns[i]

In [5]:
# Final QUBO (Risk - Return)
Q_total = {}
for (i, j), val in Q_risk.items():
    Q_total[(i, j)] = Q_total.get((i, j), 0) + val
for (i, j), val in Q_return.items():
    Q_total[(i, j)] = Q_total.get((i, j), 0) + val

# Display the final Q_total matrix
qubo_matrix = np.zeros((n, n))
for (i, j), val in Q_total.items():
    qubo_matrix[i, j] = val
    if i != j:
        qubo_matrix[j, i] = val

qubo_df = pd.DataFrame(qubo_matrix, index=selected_tickers, columns=selected_tickers)
print("\n🔧 Final QUBO Matrix (λ Σ - μ):")
print(qubo_df.round(6))


🔧 Final QUBO Matrix (λ Σ - μ):
           AAPL      AMZN     GOOGL       JPM      META      MSFT      NFLX  \
AAPL   0.004084  0.002697  0.002135  0.001682  0.002769  0.001955  0.001659   
AMZN   0.002697  0.003948  0.002909  0.002134  0.003795  0.002530  0.002169   
GOOGL  0.002135  0.002909  0.003939  0.001413  0.002904  0.001914  0.001592   
JPM    0.001682  0.002134  0.001413  0.001521  0.001816  0.001287  0.001443   
META   0.002769  0.003795  0.002904  0.001816  0.003723  0.002549  0.002381   
MSFT   0.001955  0.002530  0.001914  0.001287  0.002549  0.002019  0.001701   
NFLX   0.001659  0.002169  0.001592  0.001443  0.002381  0.001701  0.001273   
NVDA   0.003422  0.004689  0.003710  0.002428  0.004977  0.003587  0.003649   
TSLA   0.004838  0.005814  0.005178  0.003799  0.005740  0.003847  0.004016   
V      0.001385  0.001387  0.001110  0.001490  0.001436  0.000906  0.001086   

           NVDA      TSLA         V  
AAPL   0.003422  0.004838  0.001385  
AMZN   0.004689  0.005

In [6]:
from qiskit_algorithms import QAOA
from qiskit_algorithms.optimizers import COBYLA
from qiskit_algorithms.utils import algorithm_globals
from qiskit_aer.primitives import Sampler as AerSampler
from qiskit_optimization.problems import QuadraticProgram
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit.quantum_info import SparsePauliOp

In [7]:
algorithm_globals.random_seed = 42
qp = QuadraticProgram()
for i in range(n):
    qp.binary_var(name=f'x{i}')
linear = {}
quadratic = {}
for (i, j), val in Q_total.items():
    if i == j:
        linear[f'x{i}'] = val
    else:
        if (f'x{j}', f'x{i}') not in quadratic:
            quadratic[(f'x{i}', f'x{j}')] = val
qp.minimize(linear=linear, quadratic=quadratic)
num_assets_to_select = 2
qp.linear_constraint(
    linear={f'x{i}': 1 for i in range(n)},
    sense='==',
    rhs=num_assets_to_select,
    name='num_assets_constraint'
)
sampler = AerSampler()
optimizer = COBYLA(maxiter=200)
qaoa = QAOA(
    sampler=sampler,
    optimizer=optimizer,
    reps=3
)
solver = MinimumEigenOptimizer(qaoa)
result = solver.solve(qp)
print("\n✅ Optimal Portfolio Selection:")
if result.x is not None:
    selected_indices = np.where(result.x == 1)[0]
    if len(selected_indices) > 0:
        for idx in selected_indices:
            print(f"✔️ {selected_tickers[idx]} selected")
    else:
        print("No assets selected based on the optimization result (check constraints/QUBO weights)." +
              " This can happen if the optimal solution for the given QUBO and constraints is indeed all zeros.")
else:
    print("No optimal solution found or result.x is None.")
print(f"\n🧮 Binary solution: {result.x}")
print(f"🎯 Objective value: {result.fval}")


✅ Optimal Portfolio Selection:
✔️ NFLX selected
✔️ V selected

🧮 Binary solution: [0. 0. 0. 0. 0. 0. 1. 0. 0. 1.]
🎯 Objective value: 0.003206430827254917
